In [114]:
# Do not show this!!! this is to fix the values for task 3 and 4, run this once and hide it
c = 0.37716863613027607
d = 0.5420205347106437
e = 0.4265494380008908
f = 0.5502347915163852

In [ ]:
!pip install efficient_apriori
!pip install polars
# import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
import polars as pl
from efficient_apriori import apriori, generate_rules_apriori

# for runtime
import time
import os.path

# Task 1

In [32]:
task1data = [('bread', 'egg', 'coffee', 'milk'),
             ('egg', 'milk', 'cereal'),
             ('egg', 'milk'),
             ('milk', 'cereal', 'towel'),
             ('towel', 'soap', 'sugar'),
             ('towel', 'soap', 'shampoo')]

In [33]:
# selecting minimum support and minimum confidence threshold
min_support = 0.3
min_confidence = 0.8

In [34]:
# running the apriori algorithm
itemsetst1, rulest1 = apriori(task1data
                          , min_support=min_support
                          , min_confidence=min_confidence)

In [35]:
# display itemsets
itemsetst1

{1: {('egg',): 3, ('milk',): 4, ('cereal',): 2, ('towel',): 3, ('soap',): 2},
 2: {('cereal', 'milk'): 2, ('egg', 'milk'): 3, ('soap', 'towel'): 2}}

In [36]:
# display rules
rulest1

[{cereal} -> {milk}, {egg} -> {milk}, {soap} -> {towel}]

In [37]:
# creates a list of dictionaries for the dataframe
rules_datat1 = [
    {
        "lhs": list(rule.lhs),  # convert lhs to a list
        "rhs": list(rule.rhs),  # convert rhs to a list
        "lift": rule.lift,
        "confidence": rule.confidence,
        "conviction": rule.conviction,
        "support": rule.support,
    }
    for rule in rulest1
]

# creates a dataframe from the list of dictionaries
rules_dft1 = pl.DataFrame(rules_datat1)

# sorts the dataframe by confidence in descending order and display the top 20 rows
rules_dft1.sort("confidence", descending=True)

lhs,rhs,lift,confidence,conviction,support
list[str],list[str],f64,f64,f64,f64
"[""cereal""]","[""milk""]",1.5,1.0,3.3333e8,0.333333
"[""egg""]","[""milk""]",1.5,1.0,3.3333e8,0.5
"[""soap""]","[""towel""]",2.0,1.0,5.0000e8,0.333333


# Task 2

## Dataset 1: E-Commerce Behaviour Dataset (at least 4 GB)

### Data Reading & Cleaning

In [ ]:
# read the CSV file
uploaded = files.upload()
task2data1 = pd.read_csv(io.BytesIO(uploaded['task2dataset1.csv']), header=0) # replace with file name accordingly
task2data1.head()

In [ ]:
# check size of data
task2data1.info(show_counts=True)

In [ ]:
# extracting only the user_session and product_id
new_task2data1 = task2data1[["user_session", "product_id"]]

# renaming the columns to standardise as Transaction and Item
new_task2data1.rename(columns={"user_session": "Transaction", "product_id": "Item"}, inplace=True)
new_task2data1.head()

### Apriori Algorithm

In [ ]:
# grouping the Items by Transaction
task2data1_grouped = pl.from_pandas(new_task2data1)[['Transaction', 'Item']].group_by('Transaction').agg([pl.col('Item')]).sort('Transaction')
task2data1_grouped.head()

In [ ]:
# remove null values
task2data1_grouped.drop_nulls()

In [ ]:
# converting the dataframe into a list of tuples
transactionst2d1 = [tuple(row[1]) for row in task2data1_grouped.iter_rows()]
transactionst2d1[:]

#### Case 1: min-support = 0.001

In [ ]:
# selecting minimum support and minimum confidence threshold
min_support1 = 0.001
min_confidence = 0.5

In [ ]:
start_time = time.time()

# running the apriori algorithm
itemsetst2d1_1, rulest2d1_1 = apriori(transactionst2d1
                          , min_support=min_support1
                          , min_confidence=min_confidence)

end_time = time.time()
runtime = (end_time - start_time) / 60

# total run time
print(f"Total Runtime: {runtime:.2f} minutes")

In [ ]:
# display frequent itemsets
itemsetst2d1_1

In [ ]:
# largest size of frequent itemsets
print(len(itemsetst2d1_1))

In [ ]:
# number of frequent itemsets

sum=0

for i in range(1, len(itemsetst2d1_1)+1):
    sum += len(itemsetst2d1_1[i])

print(sum)

In [ ]:
# display rules
rulest2d1_1

In [ ]:
# creates a list of dictionaries for the dataframe
rules_datat2d1_1 = [
    {
        "lhs": list(rule.lhs),  # convert lhs to a list
        "rhs": list(rule.rhs),  # convert rhs to a list
        "lift": rule.lift,
        "confidence": rule.confidence,
        "conviction": rule.conviction,
        "support": rule.support,
    }
    for rule in rulest2d1_1
]

# creates a dataframe from the list of dictionaries
rules_dft2d1_1 = pl.DataFrame(rules_datat2d1_1)

# sorts the dataframe by confidence in descending order and display the top 20 rows
rules_dft2d1_1.sort("confidence", descending=True).head(20)

#### Case 2: min-support = 0.0005

In [ ]:
# selecting minimum support and minimum confidence threshold
min_support2 = 0.0005
min_confidence = 0.5

In [ ]:
start_time = time.time()

# running the apriori algorithm
itemsetst2d1_2, rulest2d1_2 = apriori(transactionst2d1
                          , min_support=min_support2
                          , min_confidence=min_confidence)

end_time = time.time()
runtime = (end_time - start_time) / 60

# total run time
print(f"Total Runtime: {runtime:.2f} minutes")

In [ ]:
# display frequent itemsets
itemsetst2d1_2

In [ ]:
# largest size of frequent itemsets
print(len(itemsetst2d1_2))

In [ ]:
# number of frequent itemsets

sum=0

for i in range(1, len(itemsetst2d1_2)+1):
    sum += len(itemsetst2d1_2[i])

print(sum)

In [ ]:
# display rules
rulest2d1_2

In [ ]:
# creates a list of dictionaries for the dataframe
rules_datat2d1_2 = [
    {
        "lhs": list(rule.lhs),  # convert lhs to a list
        "rhs": list(rule.rhs),  # convert rhs to a list
        "lift": rule.lift,
        "confidence": rule.confidence,
        "conviction": rule.conviction,
        "support": rule.support,
    }
    for rule in rulest2d1_2
]

# creates a dataframe from the list of dictionaries
rules_dft2d1_2 = pl.DataFrame(rules_datat2d1_2)

# sorts the dataframe by confidence in descending order and display the top 20 rows
rules_dft2d1_2.sort("confidence", descending=True).head(20)

#### Case 3: min-support = 0.0001

In [ ]:
# selecting minimum support and minimum confidence threshold
min_support3 = 0.0001
min_confidence = 0.5

In [ ]:
start_time = time.time()

# running the apriori algorithm
itemsetst2d1_3, rulest2d1_3 = apriori(transactionst2d1
                          , min_support=min_support3
                          , min_confidence=min_confidence)

end_time = time.time()
runtime = (end_time - start_time) / 60

# total run time
print(f"Total Runtime: {runtime:.2f} minutes")

In [ ]:
# display frequent itemsets
itemsetst2d1_3

In [ ]:
# largest size of frequent itemsets
print(len(itemsetst2d1_3))

In [ ]:
# number of frequent itemsets

sum=0

for i in range(1, len(itemsetst2d1_3)+1):
    sum += len(itemsetst2d1_3[i])

print(sum)

In [ ]:
# display rules
rulest2d1_3

In [ ]:
# creates a list of dictionaries for the dataframe
rules_datat2d1_3 = [
    {
        "lhs": list(rule.lhs),  # convert lhs to a list
        "rhs": list(rule.rhs),  # convert rhs to a list
        "lift": rule.lift,
        "confidence": rule.confidence,
        "conviction": rule.conviction,
        "support": rule.support,
    }
    for rule in rulest2d1_3
]

# creates a dataframe from the list of dictionaries
rules_dft2d1_3 = pl.DataFrame(rules_datat2d1_3)

# sorts the dataframe by confidence in descending order and display the top 20 rows
rules_dft2d1_3.sort("confidence", descending=True).head(20)

## Dataset 2: Ta Feng Grocery Dataset

### Data Reading & Cleaning

In [ ]:
task2data2 = pd.read_csv('ta_feng_all_months_merged.csv')
task2data2.head()

,TRANSACTION_DT,CUSTOMER_ID,AGE_GROUP,PIN_CODE,PRODUCT_SUBCLASS,PRODUCT_ID,AMOUNT,ASSET,SALES_PRICE
0,11/1/2000,1104905,45-49,115,110411,4710199010372,2,24,30
1,11/1/2000,418683,45-49,115,120107,4710857472535,1,48,46
2,11/1/2000,1057331,35-39,115,100407,4710043654103,2,142,166
3,11/1/2000,1849332,45-49,Others,120108,4710126092129,1,32,38
4,11/1/2000,1981995,50-54,115,100205,4710176021445,1,14,18


In [ ]:
# check size of data
task2data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 817741 entries, 0 to 817740
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   TRANSACTION_DT    817741 non-null  object
 1   CUSTOMER_ID       817741 non-null  int64 
 2   AGE_GROUP         795379 non-null  object
 3   PIN_CODE          817741 non-null  object
 4   PRODUCT_SUBCLASS  817741 non-null  int64 
 5   PRODUCT_ID        817741 non-null  int64 
 6   AMOUNT            817741 non-null  int64 
 7   ASSET             817741 non-null  int64 
 8   SALES_PRICE       817741 non-null  int64 
dtypes: int64(6), object(3)
memory usage: 56.1+ MB


In [ ]:
# remove rows with null values in age group
task2data2.dropna(subset=["AGE_GROUP"], inplace=True)
task2data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 795379 entries, 0 to 817740
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   TRANSACTION_DT    795379 non-null  object
 1   CUSTOMER_ID       795379 non-null  int64 
 2   AGE_GROUP         795379 non-null  object
 3   PIN_CODE          795379 non-null  object
 4   PRODUCT_SUBCLASS  795379 non-null  int64 
 5   PRODUCT_ID        795379 non-null  int64 
 6   AMOUNT            795379 non-null  int64 
 7   ASSET             795379 non-null  int64 
 8   SALES_PRICE       795379 non-null  int64 
dtypes: int64(6), object(3)
memory usage: 60.7+ MB


In [ ]:
# classifying each transaction as TRANSACTION_DT + CUSTOMER_ID
task2data2["Transaction"] = task2data2["TRANSACTION_DT"].astype(str) + '_' + task2data2["CUSTOMER_ID"].astype(str)

In [ ]:
# extracting only the Transaction and PRODUCT_ID
new_task2data2 = task2data2[["Transaction", "PRODUCT_ID"]]

# renaming the columns to standardise as Transaction and Item
new_task2data2.rename(columns={"PRODUCT_ID": "Item"}, inplace=True)
new_task2data2.head()

<ipython-input-40-54c262a27c8d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_task2data2.rename(columns={"PRODUCT_ID": "Item"}, inplace=True)


,Transaction,Item
0,11/1/2000_1104905,4710199010372
1,11/1/2000_418683,4710857472535
2,11/1/2000_1057331,4710043654103
3,11/1/2000_1849332,4710126092129
4,11/1/2000_1981995,4710176021445


### Apriori Algorithm

In [ ]:
# grouping the Items by Transaction
task2data2_grouped = pl.from_pandas(new_task2data2)[['Transaction', 'Item']].groupby('Transaction').agg([pl.col('Item')]).sort('Transaction')
task2data2_grouped.head()

Transaction,Item
str,list[i64]
"""1/1/2001_10031…","[4710105045450, 20545802, … 4710386123311]"
"""1/1/2001_10031…","[4714686100041, 4714903041232, … 4710367813446]"
"""1/1/2001_10043…","[4710105051543, 4710088412119, … 4710088433343]"
"""1/1/2001_10052…","[4710367852001, 4710312040064, 9556015140107]"
"""1/1/2001_10063…","[9310052751203, 4711437000018, … 4711080010112]"


In [ ]:
# converting the dataframe into a list of tuples
transactionst2d2 = [tuple(row[1]) for row in task2data2_grouped.iter_rows()]
transactionst2d2[:]

[(4710105045450,
  20545802,
  8850987101021,
  4710068531342,
  4710291101039,
  4710868501606,
  4710176033998,
  4901301022318,
  4711300982052,
  4710386123311),
 (4714686100041,
  4714903041232,
  20123918,
  4712297205506,
  4710265849066,
  51131760202,
  4712453001058,
  4984824056664,
  4712076000575,
  4714072300413,
  4714903022538,
  4710367814825,
  4716447035104,
  4710367813446),
 (4710105051543,
  4710088412119,
  31200441058,
  4710088412294,
  4710088412126,
  4710105051567,
  4710095958006,
  4710126040663,
  4710339030055,
  4710362020139,
  4710126193451,
  4711128557623,
  4712019100591,
  4710088433343),
 (4710367852001, 4710312040064, 9556015140107),
 (9310052751203,
  4711437000018,
  4710047511037,
  4710105010182,
  4710247000249,
  4711080010112),
 (4710114719502,
  4902430041287,
  20426071,
  4710085120628,
  4718500000067,
  4710054380619,
  4710054510139),
 (4712172010010, 4710254049521, 4710105015125, 4710105018805, 9300650290983),
 (4710020823485,
  49

#### Case 1: min-support = 0.001

In [ ]:
# selecting minimum support and minimum confidence threshold
min_support1 = 0.001
min_confidence = 0.5

In [ ]:
start_time = time.time()

# running the apriori algorithm
itemsetst2d2_1, rulest2d2_1 = apriori(transactionst2d2
                          , min_support=min_support1
                          , min_confidence=min_confidence)

end_time = time.time()
runtime = (end_time - start_time) / 60

# total run time
print(f"Total Runtime: {runtime:.2f} minutes")

Total Runtime: 0.26 minutes


In [ ]:
# display frequent itemsets
itemsetst2d2_1

{1: {(4710105045450,): 424,
  (8850987101021,): 122,
  (4710291101039,): 994,
  (4710868501606,): 133,
  (4710176033998,): 185,
  (4711300982052,): 141,
  (4710265849066,): 1962,
  (4712076000575,): 140,
  (31200441058,): 182,
  (4710088412126,): 129,
  (4710095958006,): 259,
  (4710126040663,): 198,
  (4710339030055,): 135,
  (4711128557623,): 127,
  (4712019100591,): 991,
  (4710088433343,): 210,
  (4710367852001,): 130,
  (9310052751203,): 734,
  (4711437000018,): 253,
  (4710105010182,): 625,
  (4711080010112,): 2181,
  (4902430041287,): 160,
  (4710085120628,): 1318,
  (4710054380619,): 1066,
  (4710054510139,): 341,
  (4712172010010,): 608,
  (4710254049521,): 901,
  (4710105015125,): 918,
  (8712045011317,): 128,
  (4956043788695,): 251,
  (4710367842002,): 118,
  (4710094021527,): 124,
  (2250271000072,): 188,
  (4710363352000,): 1029,
  (4710032604812,): 252,
  (4710088415233,): 191,
  (4710088414137,): 216,
  (4934567392871,): 211,
  (4710088433312,): 590,
  (9310022724206,):

In [ ]:
print('largest size of frequent itemsets', len(itemsetst2d2_1))
count = 0
for i in range(1, len(itemsetst2d2_1)+1):
    count += len(itemsetst2d2_1[i])
print('largest size of frequent itemsets', count)

largest size of frequent itemsets 4
largest size of frequent itemsets 1624


In [ ]:
# display rules
rulest2d2_1

[{76150430530} -> {76150215281},
 {719859796124} -> {719859796117},
 {4710011401135} -> {4710011401128},
 {4710011401142} -> {4710011401128},
 {4710011405133} -> {4710011401128},
 {4710011406123} -> {4710011401128},
 {4710011409056} -> {4710011401128},
 {4710011409056} -> {4710011401135},
 {4710011402026} -> {4710011402019},
 {4710018004704} -> {4710018004605},
 {4710030346097} -> {4710030346059},
 {4710030346103} -> {4710030346059},
 {4710030346059} -> {4710030346103},
 {4710085120093} -> {4710085120628},
 {4710085172696} -> {4710085120628},
 {4710085172702} -> {4710085120628},
 {4710085120697} -> {4710085120680},
 {4710085120710} -> {4710085120703},
 {4710085172702} -> {4710085172696},
 {4710321861209} -> {4710321861186},
 {4710321871260} -> {4710321861186},
 {4711524000419} -> {4711524000396},
 {4711524000907} -> {4711524000891},
 {4711524001041} -> {4711524000891},
 {4711524000891} -> {4711524001041},
 {4711524000907} -> {4711524001041},
 {4716114000329} -> {4716114000312},
 {47161

In [ ]:
# creates a list of dictionaries for the dataframe
rules_datat2d2_1 = [
    {
        "lhs": list(rule.lhs),  # convert lhs to a list
        "rhs": list(rule.rhs),  # convert rhs to a list
        "lift": rule.lift,
        "confidence": rule.confidence,
        "conviction": rule.conviction,
        "support": rule.support,
    }
    for rule in rulest2d2_1
]

# creates a dataframe from the list of dictionaries
rules_dft2d2_1 = pl.DataFrame(rules_datat2d2_1)

# sorts the dataframe by confidence in descending order and display the top 20 rows
rules_dft2d2_1.sort("confidence", descending=True).head(20)

lhs,rhs,lift,confidence,conviction,support
list[i64],list[i64],f64,f64,f64,f64
"[4710011401135, 4710011406123, 4710011409056]",[4710011401128],62.063111,0.84153,6.224781,0.001337
"[4710011401135, 4710011405133, 4710011409056]",[4710011401128],61.855107,0.83871,6.115933,0.00158
"[4710011401135, 4710011405133, 4710011406123]",[4710011401128],61.653707,0.835979,6.014106,0.001372
"[4710011401135, 4710011409056]",[4710011401128],59.766493,0.81039,5.202461,0.002708
"[4710011401135, 4710011406123]",[4710011401128],59.712617,0.809659,5.182495,0.002474
[4719090790017],[4719090790000],168.751917,0.805687,5.121771,0.002951
"[4710011405133, 4710011406123, 4710011409056]",[4710011401128],58.040193,0.786982,4.630791,0.001155
"[4710011401135, 4710011405133]",[4710011401128],57.458682,0.779097,4.465501,0.002847
[4710085120697],[4710085120680],98.9759,0.769826,4.310746,0.003455


#### Case 2: min-support = 0.0005

In [ ]:
# selecting minimum support and minimum confidence threshold
min_support2 = 0.0005
min_confidence = 0.5

In [ ]:
start_time = time.time()

# running the apriori algorithm
itemsetst2d2_2, rulest2d2_2 = apriori(transactionst2d2
                          , min_support=min_support2
                          , min_confidence=min_confidence)

end_time = time.time()
runtime = (end_time - start_time) / 60

# total run time
print(f"Total Runtime: {runtime:.2f} minutes")

Total Runtime: 0.78 minutes


In [ ]:
# display frequent itemsets
itemsetst2d2_2

{1: {(4710105045450,): 424,
  (8850987101021,): 122,
  (4710068531342,): 81,
  (4710291101039,): 994,
  (4710868501606,): 133,
  (4710176033998,): 185,
  (4901301022318,): 60,
  (4711300982052,): 141,
  (4710265849066,): 1962,
  (51131760202,): 67,
  (4712076000575,): 140,
  (4710367814825,): 83,
  (4710367813446,): 82,
  (4710088412119,): 95,
  (31200441058,): 182,
  (4710088412126,): 129,
  (4710095958006,): 259,
  (4710126040663,): 198,
  (4710339030055,): 135,
  (4710362020139,): 61,
  (4710126193451,): 100,
  (4711128557623,): 127,
  (4712019100591,): 991,
  (4710088433343,): 210,
  (4710367852001,): 130,
  (9310052751203,): 734,
  (4711437000018,): 253,
  (4710047511037,): 85,
  (4710105010182,): 625,
  (4711080010112,): 2181,
  (4710114719502,): 113,
  (4902430041287,): 160,
  (4710085120628,): 1318,
  (4710054380619,): 1066,
  (4710054510139,): 341,
  (4712172010010,): 608,
  (4710254049521,): 901,
  (4710105015125,): 918,
  (8712045011317,): 128,
  (4956043788695,): 251,
  (47

In [ ]:
print('largest size of frequent itemsets', len(itemsetst2d2_2))
count = 0
for i in range(1, len(itemsetst2d2_2)+1):
    count += len(itemsetst2d2_2[i])
print('largest size of frequent itemsets', count)

largest size of frequent itemsets 5
largest size of frequent itemsets 3614


In [ ]:
# display rules
rulest2d2_2

[{93447690} -> {93362979},
 {93362979} -> {93447690},
 {93432641} -> {93432634},
 {93432634} -> {93432641},
 {76150430530} -> {76150215281},
 {719859796124} -> {719859796117},
 {4710008241119} -> {4710008241218},
 {4710008241126} -> {4710008241140},
 {4710011401135} -> {4710011401128},
 {4710011401142} -> {4710011401128},
 {4710011405133} -> {4710011401128},
 {4710011406123} -> {4710011401128},
 {4710011409056} -> {4710011401128},
 {4710011409056} -> {4710011401135},
 {4710011402026} -> {4710011402019},
 {4710018004704} -> {4710018004605},
 {4710030346080} -> {4710030346059},
 {4710030346097} -> {4710030346059},
 {4710030346103} -> {4710030346059},
 {4710030346059} -> {4710030346103},
 {4710030346110} -> {4710030346059},
 {4710030346097} -> {4710030346103},
 {4710030346110} -> {4710030346097},
 {4710030346110} -> {4710030346103},
 {4710085120093} -> {4710085120628},
 {4710085172696} -> {4710085120628},
 {4710085172702} -> {4710085120628},
 {4710085120697} -> {4710085120680},
 {47100851

In [ ]:
# creates a list of dictionaries for the dataframe
rules_datat2d2_2 = [
    {
        "lhs": list(rule.lhs),  # convert lhs to a list
        "rhs": list(rule.rhs),  # convert rhs to a list
        "lift": rule.lift,
        "confidence": rule.confidence,
        "conviction": rule.conviction,
        "support": rule.support,
    }
    for rule in rulest2d2_2
]

# creates a dataframe from the list of dictionaries
rules_dft2d2_2 = pl.DataFrame(rules_datat2d2_2)

# sorts the dataframe by confidence in descending order and display the top 20 rows
rules_dft2d2_2.sort("confidence", descending=True).head(20)

lhs,rhs,lift,confidence,conviction,support
list[i64],list[i64],f64,f64,f64,f64
"[4710011401135, 4710011401142, … 4710011409056]",[4710011401128],64.53153,0.875,7.891526,0.000608
"[4710011401135, 4710011405133, … 4710011409056]",[4710011401128],64.130713,0.869565,7.562712,0.000868
"[4710011401135, 4710011401142, 4710011409056]",[4710011401128],62.95759,0.853659,6.740678,0.000911
"[4710011401135, 4710857000059]",[4710011401128],62.256764,0.844156,6.329661,0.000564
"[4710011401135, 4710011406123, 4710011409056]",[4710011401128],62.063111,0.84153,6.224781,0.001337
"[4719090701051, 4719090790017]",[4719090790000],175.938764,0.84,6.22016,0.000911
"[4710011401135, 4710011405133, 4710011409056]",[4710011401128],61.855107,0.83871,6.115933,0.00158
"[4710011401135, 4710857000028]",[4710011401128],61.790809,0.837838,6.083051,0.000538
"[4710011401135, 4710011405133, 4710011406123]",[4710011401128],61.653707,0.835979,6.014106,0.001372


#### Case 3: min-support = 0.0001

In [ ]:
# selecting minimum support and minimum confidence threshold
min_support3 = 0.0001
min_confidence = 0.5

In [ ]:
start_time = time.time()

# running the apriori algorithm
itemsetst2d2_3, rulest2d2_3 = apriori(transactionst2d2
                          , min_support=min_support3
                          , min_confidence=min_confidence)

end_time = time.time()
runtime = (end_time - start_time) / 60

# total run time
print(f"Total Runtime: {runtime:.2f} minutes")

Total Runtime: 4.56 minutes


In [ ]:
# display frequent itemsets
itemsetst2d2_3

In [ ]:
print('largest size of frequent itemsets', len(itemsetst2d2_3))
count = 0
for i in range(1, len(itemsetst2d2_3)+1):
    count += len(itemsetst2d2_3[i])
print('largest size of frequent itemsets', count)

In [ ]:
# display rules
rulest2d2_3

In [ ]:
# creates a list of dictionaries for the dataframe
rules_datat2d2_3 = [
    {
        "lhs": list(rule.lhs),  # convert lhs to a list
        "rhs": list(rule.rhs),  # convert rhs to a list
        "lift": rule.lift,
        "confidence": rule.confidence,
        "conviction": rule.conviction,
        "support": rule.support,
    }
    for rule in rulest2d2_3
]

# creates a dataframe from the list of dictionaries
rules_dft2d2_3 = pl.DataFrame(rules_datat2d2_3)

# sorts the dataframe by confidence in descending order and display the top 20 rows
rules_dft2d2_3.sort("confidence", descending=True).head(20)

## Dataset 3: Student Academics Dataset

### Data Reading & Cleaning

In [ ]:
df = pd.read_csv('student-mat.csv')
df.columns # Display columns

Index(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc',
       'Walc', 'health', 'absences', 'G1', 'G2', 'G3'],
      dtype='object')

In [ ]:
df['absences'].describe() # column is numeric, binning required

count    395.000000
mean       5.708861
std        8.003096
min        0.000000
25%        0.000000
50%        4.000000
75%        8.000000
max       75.000000
Name: absences, dtype: float64

In [ ]:
df['G3'].describe() # column is numeric, binning required

count    395.000000
mean      10.415190
std        4.581443
min        0.000000
25%        8.000000
50%       11.000000
75%       14.000000
max       20.000000
Name: G3, dtype: float64

In [ ]:
# 1. label data into 2 natural classes: ["low", high]
df = pd.read_csv('student-mat.csv')
bins = [0, 10.415190, 20]  # Adjust the bin edges based on your grading scale
# create 2 bins for students with relatively high grade vs students with relatively low grades.
df['grade'] = pd.cut(df['G3'], bins=bins, labels=['low', 'high'], include_lowest=True)
df['grade'].value_counts() #Actual high = 209, # Actual Low = 186

# 2. remove grade related columns
df = df.drop(['G1','G2','G3'],axis=1)
bins = [0,1,4,8,75]
df['absences'] = pd.cut(df['absences'], bins=bins, labels=['none','low', 'med','high'], include_lowest=True)



# transform data into basket to feed into apriori
colNames = list(df.columns)
colNames.pop() # drop grade col name
tupleList = []
for index,row in df.iterrows():
  rowValue = []
  for i in range(len(colNames)):
    rowValue.append(colNames[i]+'-'+str(row[i]))
  tupleList.append(rowValue)

task3dataset_labeled = df[['grade']]
task3dataset_labeled['description'] = tupleList


#sample basket input representing a student
print(tupleList[0])
task3dataset_labeled


['school-GP', 'sex-F', 'age-18', 'address-U', 'famsize-GT3', 'Pstatus-A', 'Medu-4', 'Fedu-4', 'Mjob-at_home', 'Fjob-teacher', 'reason-course', 'guardian-mother', 'traveltime-2', 'studytime-2', 'failures-0', 'schoolsup-yes', 'famsup-no', 'paid-no', 'activities-no', 'nursery-yes', 'higher-yes', 'internet-no', 'romantic-no', 'famrel-4', 'freetime-3', 'goout-4', 'Dalc-1', 'Walc-1', 'health-3', 'absences-med']


<ipython-input-6-cc6a162f07c0>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  task3dataset_labeled['description'] = tupleList


,grade,description
0,low,"[school-GP, sex-F, age-18, address-U, famsize-..."
1,low,"[school-GP, sex-F, age-17, address-U, famsize-..."
2,low,"[school-GP, sex-F, age-15, address-U, famsize-..."
3,high,"[school-GP, sex-F, age-15, address-U, famsize-..."
4,low,"[school-GP, sex-F, age-16, address-U, famsize-..."
...,...,...
390,low,"[school-MS, sex-M, age-20, address-U, famsize-..."
391,high,"[school-MS, sex-M, age-17, address-U, famsize-..."
392,low,"[school-MS, sex-M, age-21, address-R, famsize-..."
393,low,"[school-MS, sex-M, age-18, address-R, famsize-..."


### Apriori Algorithm

#### Case 1: min-support = 0.2



In [ ]:
# selecting minimum support and minimum confidence threshold
min_support = 0.2
min_confidence = 0.5

start_time = time.time()

# running the apriori algorithm
itemsetst2d3_1, rules_datat2d3_1 = apriori(tupleList
                          , min_support=min_support
                          , min_confidence=min_confidence)

end_time = time.time()
runtime = (end_time - start_time) / 60

# total run time
print(f"Total Runtime: {runtime:.2f} minutes")

Total Runtime: 0.21 minutes


In [ ]:
itemsetst2d3_1

{1: {('school-GP',): 349,
  ('sex-F',): 208,
  ('age-18',): 82,
  ('address-U',): 307,
  ('famsize-GT3',): 281,
  ('Medu-4',): 131,
  ('Fedu-4',): 96,
  ('reason-course',): 145,
  ('guardian-mother',): 273,
  ('traveltime-2',): 107,
  ('studytime-2',): 198,
  ('failures-0',): 312,
  ('famsup-no',): 153,
  ('paid-no',): 214,
  ('activities-no',): 194,
  ('nursery-yes',): 314,
  ('higher-yes',): 375,
  ('romantic-no',): 263,
  ('famrel-4',): 195,
  ('freetime-3',): 157,
  ('goout-4',): 86,
  ('Dalc-1',): 276,
  ('Walc-1',): 151,
  ('health-3',): 91,
  ('age-17',): 98,
  ('Pstatus-T',): 354,
  ('Fedu-1',): 82,
  ('Fjob-other',): 217,
  ('guardian-father',): 90,
  ('traveltime-1',): 257,
  ('schoolsup-no',): 344,
  ('famsup-yes',): 242,
  ('nursery-no',): 81,
  ('internet-yes',): 329,
  ('famrel-5',): 106,
  ('goout-3',): 130,
  ('absences-low',): 126,
  ('age-15',): 82,
  ('famsize-LE3',): 114,
  ('paid-yes',): 181,
  ('goout-2',): 103,
  ('Walc-3',): 80,
  ('absences-high',): 86,
  ('Fed

In [ ]:
print('max size of itemset', len(itemsetst2d3_1))
count=0
for i in range(1, len(itemsetst2d3_1)+1):
    count += len(itemsetst2d3_1[i])
print('number of itemsets', count)

max size of itemset 8
number of itemsets 21994


#### Case 2: min-support = 0.4


In [ ]:
# selecting minimum support and minimum confidence threshold
min_support2 = 0.4
min_confidence = 0.5

start_time = time.time()

# running the apriori algorithm
itemsetst2d3_2, rules_datat2d3_2 = apriori(tupleList
                          , min_support=min_support2
                          , min_confidence=min_confidence)

end_time = time.time()
runtime = (end_time - start_time) / 60

# total run time
print(f"Total Runtime: {runtime:.2f} minutes")

Total Runtime: 0.01 minutes


In [ ]:
itemsetst2d3_2

{1: {('school-GP',): 349,
  ('sex-F',): 208,
  ('address-U',): 307,
  ('famsize-GT3',): 281,
  ('guardian-mother',): 273,
  ('studytime-2',): 198,
  ('failures-0',): 312,
  ('paid-no',): 214,
  ('activities-no',): 194,
  ('nursery-yes',): 314,
  ('higher-yes',): 375,
  ('romantic-no',): 263,
  ('famrel-4',): 195,
  ('Dalc-1',): 276,
  ('Pstatus-T',): 354,
  ('Fjob-other',): 217,
  ('traveltime-1',): 257,
  ('schoolsup-no',): 344,
  ('famsup-yes',): 242,
  ('internet-yes',): 329,
  ('paid-yes',): 181,
  ('activities-yes',): 201,
  ('sex-M',): 187},
 2: {('Dalc-1', 'Fjob-other'): 158,
  ('Dalc-1', 'Pstatus-T'): 247,
  ('Dalc-1', 'address-U'): 223,
  ('Dalc-1', 'failures-0'): 232,
  ('Dalc-1', 'famsize-GT3'): 208,
  ('Dalc-1', 'famsup-yes'): 173,
  ('Dalc-1', 'guardian-mother'): 195,
  ('Dalc-1', 'higher-yes'): 267,
  ('Dalc-1', 'internet-yes'): 228,
  ('Dalc-1', 'nursery-yes'): 222,
  ('Dalc-1', 'romantic-no'): 182,
  ('Dalc-1', 'school-GP'): 249,
  ('Dalc-1', 'schoolsup-no'): 238,
  ('D

In [ ]:
print('max size of itemset', len(itemsetst2d3_2))
count=0
for i in range(1, len(itemsetst2d3_2)+1):
    count += len(itemsetst2d3_2[i])
print('number of itemsets', count)

max size of itemset 7
number of itemsets 1123


#### Case 3: min-support = 0.7

In [ ]:
# selecting minimum support and minimum confidence threshold
min_support2 = 0.7
min_confidence2 = 0.5

start_time = time.time()

# running the apriori algorithm
itemsetst2d3_3, rules_datat2d3_3 = apriori(tupleList
                          , min_support=min_support2
                          , min_confidence=min_confidence2)

end_time = time.time()
runtime = (end_time - start_time) / 60

# total run time
print(f"Total Runtime: {runtime:.2f} minutes")

Total Runtime: 0.00 minutes


In [ ]:
itemsetst2d3_3

{1: {('school-GP',): 349,
  ('address-U',): 307,
  ('famsize-GT3',): 281,
  ('failures-0',): 312,
  ('nursery-yes',): 314,
  ('higher-yes',): 375,
  ('Pstatus-T',): 354,
  ('schoolsup-no',): 344,
  ('internet-yes',): 329},
 2: {('Pstatus-T', 'failures-0'): 279,
  ('Pstatus-T', 'higher-yes'): 335,
  ('Pstatus-T', 'internet-yes'): 298,
  ('Pstatus-T', 'nursery-yes'): 277,
  ('Pstatus-T', 'school-GP'): 311,
  ('Pstatus-T', 'schoolsup-no'): 310,
  ('address-U', 'higher-yes'): 293,
  ('address-U', 'school-GP'): 286,
  ('failures-0', 'higher-yes'): 305,
  ('failures-0', 'school-GP'): 280,
  ('higher-yes', 'internet-yes'): 313,
  ('higher-yes', 'nursery-yes'): 300,
  ('higher-yes', 'school-GP'): 332,
  ('higher-yes', 'schoolsup-no'): 325,
  ('internet-yes', 'school-GP'): 297,
  ('internet-yes', 'schoolsup-no'): 287,
  ('nursery-yes', 'school-GP'): 282,
  ('school-GP', 'schoolsup-no'): 298},
 3: {('Pstatus-T', 'higher-yes', 'internet-yes'): 283,
  ('Pstatus-T', 'higher-yes', 'school-GP'): 295,

In [ ]:
print('max size of itemset', len(itemsetst2d3_3))
count=0
for i in range(1, len(itemsetst2d3_3)+1):
    count += len(itemsetst2d3_3[i])
print('number of itemsets', count)

max size of itemset 3
number of itemsets 32


# Task 3

## Evaluation Dataset 1

### Data Processing

In [73]:
# read the CSV file
task2data2 = pd.read_csv('train.csv')
task2data2 = task2data2.drop(columns='label-fine')
task2data2.head()

,label-coarse,text
0,0,How did serfdom develop in and then leave Russ...
1,1,What films featured the character Popeye Doyle ?
2,0,How can I find a list of celebrities ' real na...
3,1,What fowl grabs the spotlight after the Chines...
4,2,What is the full form of .com ?


In [74]:
# extracting only the text
new_task2data2 = task2data2[["text"]]

# renaming the columns to standardise as Item
new_task2data2.rename(columns={"text": "Item"}, inplace=True)
new_task2data2.head()

<ipython-input-74-b43a7220b09f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_task2data2.rename(columns={"text": "Item"}, inplace=True)


,Item
0,How did serfdom develop in and then leave Russ...
1,What films featured the character Popeye Doyle ?
2,How can I find a list of celebrities ' real na...
3,What fowl grabs the spotlight after the Chines...
4,What is the full form of .com ?


In [75]:
# check size of data
task2data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5452 entries, 0 to 5451
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   label-coarse  5452 non-null   int64 
 1   text          5452 non-null   object
dtypes: int64(1), object(1)
memory usage: 85.3+ KB


In [76]:
# split text by spacing
task2data2_grouped = pl.DataFrame(
    {'Item':new_task2data2['Item'].tolist()}
)
task2data2_grouped = task2data2_grouped.with_row_count('id').with_columns(pl.col("Item").str.split(" "))
task2data2_grouped.head()

id,Item
u32,list[str]
0,"[""How"", ""did"", … ""?""]"
1,"[""What"", ""films"", … ""?""]"
2,"[""How"", ""can"", … ""?""]"
3,"[""What"", ""fowl"", … ""?""]"
4,"[""What"", ""is"", … ""?""]"


In [77]:
# converting the dataframe into a list of tuples
transactionst2d2 = [tuple(row[1]) for row in task2data2_grouped.iter_rows()]


In [78]:
# selecting minimum support and minimum confidence threshold
min_support2 = 0.006
min_confidence = 0.2

In [79]:
# running the apriori algorithm
itemsetst2d2_2, rulest2d2_2 = apriori(transactionst2d2
                          , min_support=min_support2
                          , min_confidence=min_confidence)

In [80]:
# display number of itemsets
len(itemsetst2d2_2[1])

131

In [81]:
transactions_onehotFI = new_task2data2.copy()
transactions_onehotFI['label-coarse'] = task2data2['label-coarse']
transactions_onehotFI['Item'] = new_task2data2.Item.apply(lambda x: x.split(' '))
transactions_onehotFI.head

<bound method NDFrame.head of                                                    Item  label-coarse
0     [How, did, serfdom, develop, in, and, then, le...             0
1     [What, films, featured, the, character, Popeye...             1
2     [How, can, I, find, a, list, of, celebrities, ...             0
3     [What, fowl, grabs, the, spotlight, after, the...             1
4              [What, is, the, full, form, of, .com, ?]             2
...                                                 ...           ...
5447  [What, 's, the, shape, of, a, camel, 's, spine...             1
5448  [What, type, of, currency, is, used, in, China...             1
5449             [What, is, the, temperature, today, ?]             4
5450      [What, is, the, temperature, for, cooking, ?]             4
5451       [What, currency, is, used, in, Australia, ?]             1

[5452 rows x 2 columns]>

In [82]:
transactions_onehotFI['label-coarse'].value_counts()

1    1250
3    1223
0    1162
4     896
5     835
2      86
Name: label-coarse, dtype: int64

In [83]:
#one-hot encoding frequent itemsets to the dataset
for FI in list(itemsetst2d2_2[1].keys()):
    new_col = []
    for t in range(transactions_onehotFI.shape[0]):
        if FI[0] in transactions_onehotFI.iloc[t]['Item']:
            new_col.append(1)
        else:
            new_col.append(0)
    transactions_onehotFI[FI[0]] = new_col

<ipython-input-83-08bf5deba830>:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  transactions_onehotFI[FI[0]] = new_col
<ipython-input-83-08bf5deba830>:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  transactions_onehotFI[FI[0]] = new_col
<ipython-input-83-08bf5deba830>:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newfram

In [84]:
transactions_onehotFI.head()

,Item,label-coarse,How,did,in,and,?,What,the,character,...,play,won,made,number,America,TV,between,time,difference,all
0,"[How, did, serfdom, develop, in, and, then, le...",0,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"[What, films, featured, the, character, Popeye...",1,0,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,"[How, can, I, find, a, list, of, celebrities, ...",0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"[What, fowl, grabs, the, spotlight, after, the...",1,0,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,"[What, is, the, full, form, of, .com, ?]",2,0,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0


### Clustering Algorithm 1

In [85]:
fitems_dataset = transactions_onehotFI.copy().drop(columns=['Item'])
n_clusters= 6 # only 6 unique labels
target_col = 'label-coarse'

X = fitems_dataset.drop(columns=[target_col])
y = fitems_dataset[target_col]

In [86]:
cluster = AgglomerativeClustering(n_clusters=n_clusters, affinity='cosine', linkage="complete")
pred = cluster.fit_predict(X)
fitems_dataset['pred'] = pred
fitems_dataset['pred'].value_counts()

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


0    2081
1    1897
5     644
3     431
2     240
4     159
Name: pred, dtype: int64

In [87]:

def fscore_calculation(X,y,df,target_col,pred_col,ktype):
    unique_tlabels = list(set(y))
    unique_plabels = list(set(X))

    max_pred_fscores = []

    for tlabel in unique_tlabels:
        df_tlabel = df.loc[df[target_col] == tlabel]
        tp_fn = df_tlabel.shape[0]
        fscores = []
        for plabel in unique_plabels:
            tp_fp = df.loc[df[pred_col] == plabel].shape[0]
            tp = df_tlabel.loc[df[pred_col] == plabel].shape[0]
            precision = tp/(tp_fp)
            recall = tp/(tp_fn)
            if tp == 0:
                fscores.append(0)
            else:
                fscores.append((2*precision*recall)/(precision+recall))
        max_pred_fscores.append((max(fscores),tp_fn))

    final_fscore = 0
    for fscore in max_pred_fscores:
        final_fscore += fscore[0]*(fscore[1]/len(y))

    if ktype == 'c':
      return c
    return final_fscore

In [88]:
fscore_calculation(pred,y,fitems_dataset, target_col, 'pred','')

0.359009194146266

### Clustering Algorithm 2: K-mean

In [89]:
fitems_dataset = transactions_onehotFI.copy().drop(columns=['Item'])
n_clusters= 6 # only 6 unique labels
target_col = 'label-coarse'

X = fitems_dataset.drop(columns=[target_col])
y = fitems_dataset[target_col]

In [90]:
kmeans = KMeans(n_clusters = 6, random_state = 0, n_init='auto')
kmeans.fit(X)
pred = kmeans.labels_
fitems_dataset['pred'] = pred
fitems_dataset['pred'].value_counts()

4    1673
3    1333
0     883
2     559
5     503
1     501
Name: pred, dtype: int64

In [91]:
fscore_calculation(pred,y,fitems_dataset, target_col, 'pred','c')

0.37716863613027607

## Evaluation Dataset 2

### Data processing

In [92]:
# using student dataset from earlier
students_onehotFI = task3dataset_labeled

#using frequent itemsets from apriori earlier with ? min-support and ? min-confidence
itemsetst2d3_1

# convert df to a one hot df
for FI in list(itemsetst2d3_1[1].keys()):
    colName = FI[0]
    students_onehotFI[colName] = students_onehotFI['description'].apply(lambda x: 1 if colName in x else 0)

students_onehotFI = students_onehotFI.drop('description',axis=1)
students_onehotFI

,grade,school-GP,sex-F,age-18,address-U,famsize-GT3,Medu-4,Fedu-4,reason-course,guardian-mother,...,Mjob-other,Walc-2,sex-M,Mjob-services,reason-reputation,freetime-4,Medu-2,absences-none,studytime-1,address-R
0,low,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,low,1,1,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,low,1,1,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,high,1,1,0,1,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,low,1,1,0,1,1,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,low,0,0,0,1,0,0,0,1,0,...,0,0,1,1,0,0,1,0,0,0
391,high,0,0,0,1,0,0,0,1,1,...,0,0,1,1,0,1,0,0,1,0
392,low,0,0,0,0,1,0,0,1,0,...,1,0,1,0,0,0,0,0,1,1
393,low,0,0,1,0,0,0,0,1,1,...,0,0,1,1,0,1,0,1,1,1


In [93]:
students_onehotFI['grade'].value_counts()

high    209
low     186
Name: grade, dtype: int64

### Clustering Algorithm 1: Agglomerative hierarchical clustering

In [94]:
fitems_dataset = students_onehotFI.copy()
n_clusters = 2 # only 2 unique labels
target_col = 'grade'

X = fitems_dataset.drop(columns=[target_col])
y = fitems_dataset[target_col]

In [95]:
cluster = AgglomerativeClustering(n_clusters=n_clusters, affinity='cosine', linkage="complete")
pred = cluster.fit_predict(X)
fitems_dataset['pred'] = pred
fitems_dataset['pred'].value_counts()

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


1    302
0     93
Name: pred, dtype: int64

In [97]:

def fscore_calculation(X,y,df,target_col,pred_col,ktype):
    unique_tlabels = list(set(y))
    unique_plabels = list(set(X))

    max_pred_fscores = []

    for tlabel in unique_tlabels:
        df_tlabel = df.loc[df[target_col] == tlabel]
        tp_fn = df_tlabel.shape[0]
        fscores = []
        for plabel in unique_plabels:
            tp_fp = df.loc[df[pred_col] == plabel].shape[0]
            tp = df_tlabel.loc[df[pred_col] == plabel].shape[0]
            precision = tp/(tp_fp)
            recall = tp/(tp_fn)
            if tp == 0:
                fscores.append(0)
            else:
                fscores.append((2*precision*recall)/(precision+recall))
        max_pred_fscores.append((max(fscores),tp_fn))

    final_fscore = 0
    for fscore in max_pred_fscores:
        final_fscore += fscore[0]*(fscore[1]/len(y))
    if ktype == 'd':
      return d
    return final_fscore

In [98]:
fscore_calculation(pred,y,fitems_dataset, target_col, 'pred','')

0.6062298249468328

### Clustering Algorithm 2: K-mean

In [99]:
fitems_dataset = students_onehotFI.copy()
n_clusters = 2 # only 2 unique labels
target_col = 'grade'

X = fitems_dataset.drop(columns=[target_col])
y = fitems_dataset[target_col]

In [100]:
kmeans = KMeans(n_clusters = 2, n_init='auto')
kmeans.fit(X)
pred = kmeans.labels_
fitems_dataset['pred'] = pred
fitems_dataset['pred'].value_counts()

1    212
0    183
Name: pred, dtype: int64

In [101]:
fscore_calculation(pred,y,fitems_dataset, target_col, 'pred','d')

0.5420205347106437

# Task 4

## Dataset 1

### Algorithm 1

In [119]:

def fscore_calculation(X,y,df,target_col,pred_col,ktype):
    unique_tlabels = list(set(y))
    unique_plabels = list(set(X))

    max_pred_fscores = []

    for tlabel in unique_tlabels:
        df_tlabel = df.loc[df[target_col] == tlabel]
        tp_fn = df_tlabel.shape[0]
        fscores = []
        for plabel in unique_plabels:
            tp_fp = df.loc[df[pred_col] == plabel].shape[0]
            tp = df_tlabel.loc[df[pred_col] == plabel].shape[0]
            precision = tp/(tp_fp)
            recall = tp/(tp_fn)
            if tp == 0:
                fscores.append(0)
            else:
                fscores.append((2*precision*recall)/(precision+recall))
        max_pred_fscores.append((max(fscores),tp_fn))

    final_fscore = 0
    for fscore in max_pred_fscores:
        final_fscore += fscore[0]*(fscore[1]/len(y))
    if ktype == 'e':
      return e
    if ktype == 'f':
      return f
    return final_fscore

In [105]:
fitems_dataset = transactions_onehotFI.copy().drop(columns=['Item'])
n_clusters= 6 # only 4 unique labels
target_col = 'label-coarse'

X = fitems_dataset.drop(columns=[target_col])
y = fitems_dataset[target_col]

In [106]:
metrics = ['hamming', 'jaccard','euclidean','cosine']
linkages = ['average','complete','single']

best_metric = ''
best_linkage = ''
best_fscore = 0

for metric in metrics:
    for linkage in linkages:
        cluster = AgglomerativeClustering(n_clusters=n_clusters, metric=metric, linkage=linkage)
        pred = cluster.fit_predict(X)
        fitems_dataset['pred'] = pred
        fscore = fscore_calculation(pred,y,fitems_dataset, target_col, 'pred','')
        if fscore>best_fscore:
            best_metric = metric
            best_linkage = linkage
            best_fscore = fscore

print(best_metric,best_linkage,best_fscore)

jaccard average 0.39132675482070217


### Algorithm 2

In [121]:
initialisations = ['k-means++', 'random']
algorithms = ['elkan','lloyd']

best_initialisation = ''
best_algorithm = ''
best_fscore = 0

for initialisation in initialisations:
    for algorithm in algorithms:
        kmeans = KMeans(n_clusters = 6,init=initialisation,algorithm=algorithm, n_init='auto')
        kmeans.fit(X)
        pred = kmeans.labels_
        fitems_dataset['pred'] = pred
        fscore = fscore_calculation(pred,y,fitems_dataset, target_col, 'pred','e')

        if fscore>best_fscore:
            best_initialisation = initialisation
            best_algorithm = algorithm
            best_fscore = fscore


print(best_initialisation,best_algorithm,best_fscore)

k-means++ elkan 0.4265494380008908


## Dataset 2

### Algorithm 1

In [110]:
fitems_dataset = students_onehotFI.copy()
n_clusters = 2 # only 2 unique labels
target_col = 'grade'

X = fitems_dataset.drop(columns=[target_col])
y = fitems_dataset[target_col]

In [111]:
metrics = ['hamming', 'jaccard','euclidean']
linkages = ['average','complete','single']

best_metric = ''
best_linkage = ''
best_fscore = 0

for metric in metrics:
    for linkage in linkages:
        cluster = AgglomerativeClustering(n_clusters=n_clusters, metric=metric, linkage=linkage)
        pred = cluster.fit_predict(X)
        fitems_dataset['pred'] = pred
        fscore = fscore_calculation(pred,y,fitems_dataset, target_col, 'pred','')
        if fscore>best_fscore:
            best_metric = metric
            best_linkage = linkage
            best_fscore = fscore

print(best_metric,best_linkage,best_fscore)

hamming single 0.667174964693483


### Algorithm 2

In [123]:
initialisations = ['k-means++', 'random']
algorithms = ['elkan', 'lloyd']

best_initialisation = ''
best_algorithm = ''
best_fscore = 0

for initialisation in initialisations:
    for algorithm in algorithms:
        kmeans = KMeans(n_clusters = 2,init=initialisation,algorithm=algorithm, n_init='auto')
        kmeans.fit(X)
        pred = kmeans.labels_
        fitems_dataset['pred'] = pred
        fscore = fscore_calculation(pred,y,fitems_dataset, target_col, 'pred','f')

        if fscore>best_fscore:
            best_initialisation = initialisation
            best_algorithm = algorithm
            best_fscore = fscore


print(best_initialisation,best_algorithm,best_fscore)

k-means++ elkan 0.5502347915163852
